## This is a script to solve for the motion of a massive rod double pendulum using DCA

#### Imports

In [1]:
import numpy as np
import math
from scipy.integrate import odeint
import matplotlib.pyplot as plt

import MBstructs as MB
import MultiBodyFuncts as MBF
import DCA

%matplotlib inline

#### Kinematic Sweep

In [2]:
# Kinematic sweep is trivial for planar n-link pendulum 
def kinematics(bodies,q,u):
    for body,theta,omega in zip(bodies,q,np.cumsum(u)):
        body.omega = omega
        body.C0 = MBF.DCM(theta)

In [3]:
def simulate(q,u,tspan,n,joints,BC1,BC2):
    '''
    his function extracts the generalized coordinates from the solution 
    of the equations of motion after calling the DCA to solve the equations of motion 
    '''
    
    # Get the Kinematics
    kinematics(bodies,q,u)
    
    # All physical properties of the rigid bodies are the same
    m = 1
    l = 1
    I = 1/12*m*l**2
    bodiesRIGID = [MB.Rigid_Body(m,l,I) for i in range(n)]
    

    
    #Call the Recursive DCA Algorithm
    #This returns a list of the form:
    #[A11,A12,A21,A22,...,An1,An2]
    #where Axy corresponds to the acceleration
    #of the yth handle of the xth body
    ll=DCA.openR(n,0,bodies,joints,BC1,BC2)

    #loop to fill d_dt with the acceleration values
    d_dt=np.zeros((2*n))
    for j in range(0,n):
        if j == 0:
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(jjs[j].P),A1)
        else:
            A2= ll.pop(0)
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(jjs[j].P),(A1-A2))
    
    #add the velocities to d_dt and return to the integrator
    d_dt[:n]=state[n:]
    return d_dt 

#### System specifications and Initial Conditions

In [4]:
# Specify number of bodies
n = 10

# q1 and q2 are measured as the angle the rod makes with -n2 axis
q0 = np.zeros((n))
q0[0] = np.pi/2

u0 = np.zeros((n))
state0 = np.hstack((q0,u0))

# Length of time of the simulation
tspan = np.arange(0,5,.01)

#### System Initialization

In [3]:
def simulate(q,u,tspan,n,joints,BC1,BC2):
    '''
    his function extracts the generalized coordinates from the solution 
    of the equations of motion after calling the DCA to solve the equations of motion 
    '''
    
    # Get the Kinematics
    kinematics(bodies,q,u)
    
    # All physical properties of the rigid bodies are the same
    m = 1
    l = 1
    I = 1/12*m*l**2
    bodiesRIGID = [MB.Rigid_Body(m,l,I) for i in range(n)]
    

    
    #Call the Recursive DCA Algorithm
    #This returns a list of the form:
    #[A11,A12,A21,A22,...,An1,An2]
    #where Axy corresponds to the acceleration
    #of the yth handle of the xth body
    ll=DCA.openR(n,0,bodies,joints,BC1,BC2)

    #loop to fill d_dt with the acceleration values
    d_dt=np.zeros((2*n))
    for j in range(0,n):
        if j == 0:
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(jjs[j].P),A1)
        else:
            A2= ll.pop(0)
            A1=ll.pop(0)
            d_dt[j+n]=np.dot(np.transpose(jjs[j].P),(A1-A2))
    
    #add the velocities to d_dt and return to the integrator
    d_dt[:n]=state[n:]
    return d_dt 

#### Simulate it

In [6]:
# odeint is the numerical integrator used
yy=odeint(simulate,state0,tspan,(n,joints,2,1))

AttributeError: 'Rigid_Body' object has no attribute 'C0'

#### Check Results and Plot

In [ ]:
# The energy of the system is calculated and plotted
energy=MBF.PendEnergy(yy,bs)
KE=energy[:,0]
PE=energy[:,1]
TE=energy[:,2]

plt.plot(Time,TE-TE[0])
plt.xlabel("Time [s]")
plt.ylabel("energy")
plt.title("System Energy")
plt.show()

plt.plot(Time,PE,Time,KE)
plt.xlabel("Time[s]")
plt.ylabel("energy")
plt.title("Kinetic and Potential Energy")
plt.show

In [ ]:
# Pot Generalized Coordinates
# plt.plot(Time,yy[:,:n])
# plt.xlabel("Time [s]")
# plt.ylabel("Generalized Coordinates [Rad]")
# plt.title("System Response")

# plt.show()

# plt.plot(Time,yy[:,n:])
# plt.xlabel(("Time[s]"))
# plt.ylabel(("Generalized Speeds [Rad/s]"))
# plt.title("System Response")

# plt.show()